In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('dags')
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.engine import URL, Engine

# Replace the placeholders with your PostgreSQL database credentials
username = 'airflow-assessment-user'
password = '6tDojhrdmJ'
host = 'localhost'
port = 5433
database = 'airflow-assessment'


In [3]:


# DATABASE_URL = f"postgresql+psycopg2://{conn_details.user}:{conn_details.password}@{conn_details.host}:{conn_details.port}/{conn_details.schema}"
AIRFLOW_SCHEMA_NAME = "airflow-assessment"


url =  URL.create(
    drivername="postgresql+psycopg2",
    username=username,
    password=password,
    host=host,
    port=port,
    database=database,
)

engine = create_engine(url)
engine.connect()

In [17]:
df = pd.read_json('http://127.0.0.1/cdm-api/companies')
df.ibans = df.ibans.apply(pd.eval)
df

,id,ibans,name,address
0,1,[NL58EQEN7258967872],Royal Schagen,Sarina-Nette-Allee 2/8\n08025 Hannoversch Münden
1,2,[NL72CVMA8072684711],Harris Group,308 Aaron isle\nStokesfurt\nN99 4NR
2,3,[DE45306993503114243904],Schönland,Julietteboulevard 915\n1962GQ\nUlft
3,4,[GB34HTSZ21460928267258],Dupont,Steinbergstr. 44\n27910 Parchim
4,5,[FR7209925764719308621072695],Croon Elektrotechniek,Tlustekstraße 5\n55633 Bremervörde
...,...,...,...,...
95,96,[NL73QLRU1042042324],Oda & Freshour,Taraboulevard 9\n2226 XI\nZweins
96,97,[GB56BYQM16792331709055],Mitschke,Eckhart-Renner-Allee 757\n17504 Kassel
97,98,[DE36681676155117771996],Marchand S.A.S.,Annaring 464\n1397 OE\nErm
98,99,[GB51SJEV83747591864014],Kostolzin AG,"8, chemin de Delannoy\n02547 Étienne"


In [18]:

df = df.explode('ibans')
df


,id,ibans,name,address
0,1,NL58EQEN7258967872,Royal Schagen,Sarina-Nette-Allee 2/8\n08025 Hannoversch Münden
1,2,NL72CVMA8072684711,Harris Group,308 Aaron isle\nStokesfurt\nN99 4NR
2,3,DE45306993503114243904,Schönland,Julietteboulevard 915\n1962GQ\nUlft
3,4,GB34HTSZ21460928267258,Dupont,Steinbergstr. 44\n27910 Parchim
4,5,FR7209925764719308621072695,Croon Elektrotechniek,Tlustekstraße 5\n55633 Bremervörde
...,...,...,...,...
95,96,NL73QLRU1042042324,Oda & Freshour,Taraboulevard 9\n2226 XI\nZweins
96,97,GB56BYQM16792331709055,Mitschke,Eckhart-Renner-Allee 757\n17504 Kassel
97,98,DE36681676155117771996,Marchand S.A.S.,Annaring 464\n1397 OE\nErm
98,99,GB51SJEV83747591864014,Kostolzin AG,"8, chemin de Delannoy\n02547 Étienne"


In [20]:
df.ibans.str.slice(0, 2)

0     NL
1     NL
2     DE
3     GB
4     FR
      ..
95    NL
96    GB
97    DE
98    GB
99    NL
Name: ibans, Length: 100, dtype: object

In [4]:
import os
os.environ['_AIRFLOW__AS_LIBRARY'] = '1'
x = os.environ['_AIRFLOW__AS_LIBRARY']
bool(x)

True

In [6]:
from airflow_assessment.database import create_schema_if_not_exists_with_engine, create_table_if_not_exists
create_schema_if_not_exists_with_engine(engine, AIRFLOW_SCHEMA_NAME)

In [ ]:
from airflow_assessment.database import create_tables
create_tables(engine)

In [12]:
from airflow_assessment.models.alchemy import CompanyModel


CompanyModel.__table_args__['schema']

'airflow-assessment'

In [13]:
from airflow_assessment.models.alchemy import CompanyModel


create_table_if_not_exists(engine, model=CompanyModel)

In [15]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [27]:
from datetime import datetime
from airflow_assessment.ingest import get_transactions_from_api_interval


get_transactions_from_api_interval(start_date=datetime(2021, 1, 1), url='http://127.0.0.1/cdm-api/transactions/swift')

INFO:airflow_assessment.ingest:Retrieving SEPA transactions from API for 2021-01-01 00:00:00


ValueError: day is out of range for month

In [29]:
from airflow_assessment.utils import last_day_of_month


last_day_of_month(datetime(2023, 1, 1, 4, 5))

datetime.datetime(2023, 1, 31, 4, 5)